In [1]:
using Distributions
type QueueCustomer
    time::Float64  #When do I arrrive
    service_time::Float64 #How much time it take to process my request
    wait_time::Float64 #How much time I spent waiting in the queue
    ID::Int
end
function customer_queue_gen(numCustomers,λ,a,b)
    #a,b are the parameters of the uniform distribution (min, max)
    t=0.0
    interarrival_dist=Exponential(λ)
    proccessing_dist=Uniform(a,b) #considering that the service time follows a uniform distribution
    function f(i)  #Our closure that will be used in the returned generator
        next_custmer_at=rand(interarrival_dist)
        t+=next_custmer_at
        service_time=rand(proccessing_dist)
        qc=QueueCustomer(t,service_time,Inf,i)
    end
    (f(i) for i=1:numCustomers) #The returned generator
end

customer_queue_gen (generic function with 1 method)

In [2]:
function service_processing_c_stations_gen(queue_gen::Base.Generator,c)
    t_next_customer_at=zeros(c)
    wait_time_list=Float64[]
    service_time_list=Float64[]
    arrival_time_list=Float64[]
    for qc in queue_gen
        current_station=indmin(t_next_customer_at)
        t_next_customer_at[current_station]=max(t_next_customer_at[current_station],qc.time)
        wait_time=t_next_customer_at[current_station]-qc.time
        push!(wait_time_list,wait_time)
        push!(service_time_list,qc.service_time)
        push!(arrival_time_list,qc.time)
        t_next_customer_at[current_station]+=qc.service_time
    end
    wait_time_list, service_time_list,arrival_time_list
end

service_processing_c_stations_gen (generic function with 1 method)

In [4]:
#calling the function witn λ = 2.33, service rate follows a uniform distribution with a=3 and b=8 and there's 3 servers
que_gn=customer_queue_gen(1000000,2.33,3,8) 
wait,serve,arriv=service_processing_c_stations_gen(que_gn,3)
lastArrival= arriv[end]
lastService = serve[end]
lastWaiting = wait[end]
sumServices = sum(serve)
endtime = lastArrival+lastService +lastWaiting
utilization= sumServices/endtime

println("max waiting time:",maximum(wait))
println("max service time:",maximum(serve))
println("max arrival time:",maximum(arriv))
println("Arrival time of the last customer:",lastArrival,"Service time of the last customer :",lastService ,"Waiting time of the last customer:",lastWaiting)
println("end time:", endtime)
println("Sum of all Service Times: ",sumServices)
println("utilization: ",utilization)


max waiting time:44.91895520710386
max service time:7.999992262934496
max arrival time:2.333042254089105e6
Arrival time of the last customer:2.333042254089105e6Service time of the last customer :6.395240138200441Waiting time of the last customer:11.591007404960692
end time:2.333060240336648e6
Sum of all Service Times: 5.498796314693465e6
utilization: 2.356902843580249
